# Clean Private Property Data
In this notebook, we perform basic data cleaning and feature engineering on [URA private property caveat data](https://www.ura.gov.sg/realEstateIIWeb/transaction/search.action). I manually searched and saved data on all districts and all propety types separately, because the URA Caveat search facility limits the type of query you can make at any one time.

In [1]:
# Import required modules
import lightgbm as lgbm
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import warnings

from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import RepeatedKFold, KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor, Booster

# Settings
warnings.filterwarnings('ignore')

## Import Data

In [2]:
# Get files
file_list = os.listdir('../Data/Private/')

# Initialise dataframe
df = pd.DataFrame()

for file in file_list:
    
    # Import data
    temp_df = pd.read_csv('../Data/Private/'+file, skiprows=1)
    
    # Get last row
    last_row = int(temp_df['S/N'][temp_df['S/N'].str.contains('record\(s\)').fillna(False)].values[0].split(' ')[0])
    
    if 'district' in file:
        temp_df['category'] = 'Landed'
    elif 'strata' in file:
        temp_df['category'] = 'Landed'
    elif 'condo' in file:
        temp_df['category'] = 'NonLanded'
    elif 'exec' in file:
        temp_df['category'] = 'NonLanded'
    
    # Append
    df = pd.concat([df, temp_df.iloc[:last_row]])

## Data Cleaning
The data was relatively clean if not for the `tenure` feature. I performed the following steps for cleaning `tenure`:

1. Missing entries for Tenure: Filled in the information using a simple Google search on the relevant properties.
2. Tenure without start year: I found out that these were properties that were still under construction. I filled in the completion year for these properties through Google.
3. Freehold properties had no starting year for tenure: I made the assumption that all Freehold properties were constructed in 1985. On hindsight, this could have affected C-Value’s accuracy.

With `tenure` cleaned, I was able to generate a new feature: `age`.

In [3]:
# Reset index
df = df.reset_index(drop=True)

# Fill in known entries
df['Tenure'].loc[15579] = '999 yrs lease commencing from 2014'
df['Tenure'].loc[28532] = 'Freehold'
df['Tenure'].loc[56924] = 'Freehold'
df['Tenure'].loc[58533] = '999 yrs lease commencing from 1988'
df['Tenure'].loc[58537] = 'Freehold'
df['Tenure'][df['Project Name'] == 'PIERMONT GRAND'] = '99 yrs lease commencing from 2023'
df['Tenure'] = df.Tenure.str.replace('27/03/2018', '2018')

# Calculate remaining lease
lease_years = df.Tenure.apply(lambda x: x.strip().split(' ')[0])
lease_years[lease_years == 'Freehold'] = 9999
lease_years = pd.to_numeric(lease_years)
lease_years[lease_years > 999] = 9999

start_year = df.Tenure.apply(lambda x: x.strip().split(' ')[-1])
start_year[start_year == 'Freehold'] = 2019
start_year = start_year.astype(int)

df['remaining_lease'] = start_year + lease_years - 2019
df['freehold'] = (df['Tenure'] == 'Freehold').astype(int)

# Age
df['age'] = 1985
df['age'][df.Tenure != 'Freehold'] = start_year[df.Tenure != 'Freehold']
df['age'][df.age == ''] = 1985
df['age'] = 2019-pd.to_numeric(df.age)

# Clean district
df['Postal District'] = pd.to_numeric(df['Postal District']).astype(int)

# Rename columns
df.columns = df.columns.str.replace('[^a-zA-Z0-9\s_]', '').str.strip().str.lower().str.replace(' ', '_')
df = df.rename(columns={'unit_price_psf': 'target'})

## Export Data

In [4]:
# Save data
df.to_csv('../Data/caveat_data_final.csv', index=False)